In [ ]:
import pandas as pd

ruta_duplicados = 'Data consolidada/df_nombres_duplicados.xlsx'
df_duplicados = pd.read_excel(ruta_duplicados)

#DICCIONARIO LINK - NUEVO NOMBRE
ruta_diccionario_nombres_duplicados = 'Data consolidada/diccionario_nombres repetidos.xlsx'
df_diccionario  = pd.read_excel(ruta_diccionario_nombres_duplicados)

#del df_nombres_modificados['Nombre Producto']
# df_nombres_modificados.rename([])

# ruta_consolidados = 'Data consolidada/consolidado.xlsx'
# df_consolidados = pd.read_excel(ruta_consolidados)

# df_consolidados = df_consolidados.drop_duplicates(
#     subset=['Nombre Producto'],
#     keep='first'
# )
